In [1]:
# definitions up front
!which python
#pip install openeo
import openeo

/home/pzellner@eurac.edu/.conda/envs/eurac_env/bin/python


In [2]:
# connect to backend
eodc_conn = openeo.connect("https://openeo-dev.eodc.eu/").authenticate_basic("openeo-user", "imSLLCd6rGT8")
eodc_conn

<Connection to 'https://openeo-dev.eodc.eu/v1.0' with BearerAuth>

In [29]:
# list collections
eodc_conn.list_processes()

[{'categories': ['math'],
  'deprecated': False,
  'description': 'Computes the absolute value of a real number `x`, which is the "unsigned" portion of x and often denoted as *|x|*.\n\nThe no-data value `null` is passed through and therefore gets propagated.',
  'examples': [{'arguments': {'x': 0}, 'returns': 0},
   {'arguments': {'x': 3.5}, 'returns': 3.5},
   {'arguments': {'x': -0.4}, 'returns': 0.4},
   {'arguments': {'x': -3.5}, 'returns': 3.5}],
  'exceptions': {},
  'experimental': False,
  'id': 'absolute',
  'links': [{'href': 'http://mathworld.wolfram.com/AbsoluteValue.html',
    'rel': 'about',
    'title': 'Absolute value explained by Wolfram MathWorld'}],
  'parameters': [{'deprecated': False,
    'description': 'A number.',
    'experimental': False,
    'name': 'x',
    'optional': False,
    'schema': {'minItems': 0.0, 'type': ['number', 'null']}}],
  'process_graph': {'if': {'arguments': {'accept': {'from_node': 'multiply'},
     'reject': {'from_parameter': 'x'},
     'value': {'from_node': 'lt'}},
    'process_id': 'if',
    'result': True},
   'lt': {'arguments': {'x': {'from_parameter': 'x'}, 'y': 0},
    'process_id': 'lt'},
   'multiply': {'arguments': {'x': {'from_parameter': 'x'}, 'y': -1},
    'process_id': 'multiply'}},
  'returns': {'description': 'The computed absolute value.',
   'schema': {'minItems': 0.0, 'minimum': 0.0, 'type': ['number', 'null']}},
  'summary': 'Absolute value'},
 {'categories': ['math'],
  'deprecated': False,
  'description': 'Sums up the two numbers `x` and `y` (*x + y*) and returns the computed sum.\n\nNo-data values are taken into account so that `null` is returned if any element is such a value.\n\nThe computations follow [IEEE Standard 754](https://ieeexplore.ieee.org/document/8766229) whenever the processing environment supports it.',
  'examples': [{'arguments': {'x': 5, 'y': 2.5}, 'returns': 7.5},
   {'arguments': {'x': -2, 'y': -4}, 'returns': -6},
   {'arguments': {'x': 1, 'y': None}, 'returns': None}],
  'exceptions': {},
  'experimental': False,
  'id': 'add',
  'links': [{'href': 'http://mathworld.wolfram.com/Sum.html',
    'rel': 'about',
    'title': 'Sum explained by Wolfram MathWorld'},
   {'href': 'https://ieeexplore.ieee.org/document/8766229',
    'rel': 'about',
    'title': 'IEEE Standard 754-2019 for Floating-Point Arithmetic'}],
  'parameters': [{'deprecated': False,
    'description': 'The first summand.',
    'experimental': False,
    'name': 'x',
    'optional': False,
    'schema': {'minItems': 0.0, 'type': ['number', 'null']}},
   {'deprecated': False,
    'description': 'The second summand.',
    'experimental': False,
    'name': 'y',
    'optional': False,
    'schema': {'minItems': 0.0, 'type': ['number', 'null']}}],
  'process_graph': {'sum': {'arguments': {'data': [{'from_parameter': 'x'},
      {'from_parameter': 'y'}],
     'ignore_nodata': False},
    'process_id': 'sum',
    'result': True}},
  'returns': {'description': 'The computed sum of the two numbers.',
   'schema': {'minItems': 0.0, 'type': ['number', 'null']}},
  'summary': 'Addition of two numbers'},
 {'categories': ['cubes'],
  'deprecated': False,
  'description': 'Adds a new named dimension to the data cube.\n\nAfterwards, the dimension can be referenced with the specified `name`. If a dimension with the specified name exists, the process fails with a `DimensionExists` error. The dimension label of the dimension is set to the specified `label`.',
  'exceptions': {'DimensionExists': {'http': 400,
    'message': 'A dimension with the specified name already exists.'}},
  'experimental': False,
  'id': 'add_dimension',
  'parameters': [{'deprecated': False,
    'description': 'A data cube to add the dimension to.',
    'experimental': False,
    'name': 'data',
    'optional': False,
    'schema': {'minItems': 0.0, 'subtype': 'raster-cube', 'type': 'object'}},
   {'deprecated': False,
    'description': 'Name for the dimension.',
    'experimental': False,
    'name': 'name',
 

In [3]:
# spatial subset (east of brunico)
# west  = 11.99707 
# east  = 12.48047
# south = 46.64338 
# north = 46.86753

# spatial point welschenhofen forrest disturbance
west  = 11.53818
east  = 11.53845
south = 46.48657 
north = 46.48676

bbox = [west,south,east,north]
print('Area Of Interest:','\nwest',bbox[0],'\neast',bbox[2],'\nsouth',bbox[1],'\nnorth',bbox[3])

Area Of Interest: 
west 11.53818 
east 11.53845 
south 46.48657 
north 46.48676


In [6]:
# define process graph
from openeo.processes import array_element, normalized_difference

collection      = 'boa_sentinel_2'
spatial_extent  = {'west':bbox[0],'east':bbox[2],'south':bbox[1],'north':bbox[3]}
temporal_extent = ["2016-06-01", "2017-06-01"]
bands           = ["B04","B08"]

s2_boa = eodc_conn.load_collection(collection,spatial_extent=spatial_extent,bands=bands,temporal_extent=temporal_extent)

#s2_boa_repr = reproject doesn't exist

#l2a_force = s2_boa.save_result(format="netCDF")

ndvi = s2_boa.reduce_dimension(dimension="bands",reducer = lambda value: (array_element(value,1)-array_element(value,0))/(array_element(value,1)+array_element(value,0)))
#ndvi = s2_boa.ndvi(nir = "B08", red = "B04", target_band = "NDVI")
result = ndvi.save_result(format="netCDF")




In [7]:
import json
print(json.dumps(result.graph,indent=2))

{
  "loadcollection1": {
    "process_id": "load_collection",
    "arguments": {
      "bands": [
        "B04",
        "B08"
      ],
      "id": "boa_sentinel_2",
      "spatial_extent": {
        "west": 11.53818,
        "east": 11.53845,
        "south": 46.48657,
        "north": 46.48676
      },
      "temporal_extent": [
        "2016-06-01",
        "2017-06-01"
      ]
    }
  },
  "reducedimension1": {
    "process_id": "reduce_dimension",
    "arguments": {
      "data": {
        "from_node": "loadcollection1"
      },
      "dimension": "bands",
      "reducer": {
        "process_graph": {
          "arrayelement1": {
            "process_id": "array_element",
            "arguments": {
              "data": {
                "from_parameter": "data"
              },
              "index": 1
            }
          },
          "arrayelement2": {
            "process_id": "array_element",
            "arguments": {
              "data": {
                "from_paramete

In [8]:
# create job
job_eodc_s2 = eodc_conn.create_job(result.graph)
job_id_eodc_s2 = job_eodc_s2.job_id
if job_id_eodc_s2:
    print("Batch job created with id: ",job_eodc_s2)
    job_eodc_s2.start_job()
else:
    print("Error! Job ID is None")

Batch job created with id:  <RESTJob job_id='jb-a4026502-8dec-457e-9aae-a17b83a89757'>


In [10]:
# check status of job
eodc_conn = openeo.connect("https://openeo-dev.eodc.eu/").authenticate_basic("openeo-user", "imSLLCd6rGT8")
job_eodc_s2 = eodc_conn.job(job_id_eodc_s2)
job_description = job_eodc_s2.describe_job()
print("Batch job with id: ",job_id_eodc_s2, ' is ',job_description['status'])

Batch job with id:  jb-a4026502-8dec-457e-9aae-a17b83a89757  is  running


In [11]:
# download result
results_eodc = job_eodc_s2.get_results()
results_eodc.download_file("./s2_eodc_ndvi.nc")

OpenEoApiError: [500] 500: 'dict' object has no attribute 'startswith'